In [ ]:
import os
import re
from email.parser import Parser
import pandas as pd 
import datetime


rootdir = "/Users/sarahpursley/Downloads/maildir"  # full Enron email directory

emails = []
for direct, sub_direct, filenames in os.walk(rootdir): # only look at sent emails (matching action of SENT spam)
    for s in sub_direct:
        if s == 'sent':
            path = direct + "/" + s
            for file in os.listdir(path):
                os.chdir(path)
                f = open(file, "r")
                
                try:
                    f = f.read()
                    email = Parser().parsestr(f)
                    emails.append(email)
                    
                except UnicodeDecodeError:
                    continue
                    
print(len(emails))

In [ ]:
def get_content(full_email):
    content = []
    final = []
    s = " "
    for e in full_email:
        if e != '': 
            content.append(e)

    for c in content:
        if '<' in c or '@' in c or ':' in c or '-' in c:
            continue
        else:
            final.append(c)
    return s.join(final).replace("  ", " ")

email_dict = {}

for email in emails:

    for key, value in email.items():
        if key in email_dict:
            email_dict[key].append(value)
        else:
            email_dict[key] = [value]
    
    full_email = email.get_payload(decode = True).decode("utf-8") 
    full_email = full_email.splitlines()
    email_contents = get_content(full_email)
    
    if 'content' in email_dict:
        email_dict['content'].append(email_contents)

    else:
        email_dict['content'] = [email_contents]

In [ ]:
def get_days(dates):
    day_dict = {}
    
    for d in dates:
        d = d.replace(",", "").split(" ")
        day_of_week = d[0]
        if day_of_week in day_dict:
            day_dict[day_of_week] += 1
        else:
            day_dict[day_of_week] = 1
        #day_list.append(day_of_week)
    
    return day_dict

def get_domains(emails):
    all_domains = []  

    for email in emails:
        if len(email.split()) > 1:
            continue
        else:
            split = email.split("@")
            domain = split[1]
        
            if domain not in all_domains:
                all_domains.append(domain)

    return all_domains 

In [ ]:
dates = email_dict['Date']
day_week = get_days(dates)

df_day = pd.DataFrame.from_dict(day_week, orient= 'index', columns = ['Freq'])
df_result = df_day.sort_values(['Freq'], ascending = False)

score = range(0, len(df_day['Freq']))

df_result['Score'] = score
df_result


In [ ]:
# check if in valid_domains 
# proxy for a theoretical total email database - check if sender is legitimate
domain_comb = email_dict['To'] + email_dict['From']
valid_domains = get_domains(domain_comb)
len(valid_domains)

In [ ]:
import numpy as np
content = email_dict['content']
content2 = content.copy()
content_char = []
for con in content2:
    content_len = len(con.split())
    if content_len != 0:
        content_char.append(content_len)
   #print(f"Approx {content_len} characters long.")

content_char = list(set(content_char))
content_char

split_len = np.array_split(content_char, 50)
split_len

In [ ]:
# USE SAME CAPITALIZATION RATIO FROM STYLOMETRICS
# counts number of capital letters and periods in a string and returns the ratio

def count_period_ratio(string):
    """Input is a string of article"""
   # print(string)
    period_cnt = 0
    capital_cnt = 0
    
    for letter in string:
        if letter == ".":
            period_cnt+=1
        elif letter.isupper() == True:
            capital_cnt += 1
    
    ratio=capital_cnt/period_cnt
    return ratio

content = email_dict['content']
content_dict = {}
i = 0

for c in content:
    content_dict[i] = [c, len(c.strip().lstrip())]
    i += 1

for key,value in content_dict.items():
    try:
        ratio = count_period_ratio(value[0].strip().lstrip())
        content_dict[key].append(ratio)
    except ZeroDivisionError:
        content_dict[key].append(0)
        continue


In [ ]:
ratearray = content_dict.values()

df1= pd.DataFrame(ratearray)
df1.columns=["enron_text", "content_len", "ratio"]
df1['norm_ratio'] = df1['ratio']/df1['content_len']
df1



In [ ]:
import json 
#os.listdir(".")
f = open('fraudulent_emails_relationship.json') 
messages = json.load(f)
messages

In [ ]:
enron_ratios = df1.groupby(by=["content_len"]).mean().to_dict()
enron_avg_length = round(np.mean(content_char))
enron_stdev = round(np.std(content_char))

def closest_val(num):
    func = lambda content_char : abs(content_char - num)
    closest = min(content_char, key = func)
    if closest == None:
        return 0
    else:
        return closest

def same_val(num):
    num1 = closest_val(num)
    num2 = num1 in content_char
    
    if num2 == True:
        return "enron email exists with this length"
    else:
        return "enron email does not exist with this length"

def match_val(num):
    normalized = enron_ratios['norm_ratio']
    for key, val in normalized.items():
        if key != None and num != None:
            if key == num:
                return val 
            else:
                return 0


In [ ]:
import datetime
score_list = df_result.to_dict()['Score']
score_list[None] = None 
valid_domains = get_domains(domain_comb)


def match_dates(fraud_date_format):
    
    day_match = {
        'Sun': 'Sunday',
        'Mon': 'Monday',
        'Tue': 'Tuesday',
        'Wed': 'Wednesday',
        'Thu': 'Thursday',
        'Fri': 'Friday',
        'Sat': 'Saturday'}
    
    fraud_date = fraud_date_format.rpartition('T')[0]
    day_of_week = datetime.date(year,month,day)
    full_day = day_of_week.strftime("%A")
    
    #matched_day
    for key,value in day_match.items():
        if value == full_day:
            return key 
        
def domain_lookup(domain_list,domain):
    for vd in domain_list:
        if domain == vd:
            return 'found'
        else:
            return 'not found'
        
# HIGHER SCORE = HIGHER LIKELIHOOD OF SENDING ON A NON FREQUENT DAY
for message in messages:
    content = message["X-TIKA:content"].lstrip().strip()
    #content_len = len(content.split())
    closest_enron_value = closest_val(len(content.split()))
    closest_enron_ratio = match_val(closest_enron_value)
    length_exists = same_val(closest_enron_value)
    
    try:
        author_email = message['Message:From-Email']
        domain = author_email.split('@')[-1]
        domain_match = domain_lookup(valid_domains, domain)
    except KeyError:
        author_email = None

    
    try:
        date_created = match_dates(message['Creation-Date'])
        score_created = score_list[date_created]
    except KeyError:
        date_created = None
        score_created = None
        
    try: 
        date_sent = message['MboxParser-from'].strip("  ").split(" ")[2]
        score_sent = score_list[date_sent]
    except KeyError:
        date_sent = None
        score_sent = None
        
    message['fraud msg characters'] = len(content.split())
    message['enron avg. msg characters'] = int(enron_avg_length)
    message['enron find similar characters'] = length_exists
    message['enron closest capitilization ratio'] = closest_enron_ratio
    message['enron capitilzation ratio difference'] = message['capitalization_ratio'] - closest_enron_ratio
    message['enron domain lookup'] = (domain, domain_match)
    message['enron day created score'] = (date_created, score_created)
    message['enron day sent score'] = (date_sent, score_sent)
    

In [ ]:
messages

In [ ]:
with open('fraudulent_emails_relationship_enrondata.json', 'w') as outfile:
    json.dump(messages, outfile)

In [ ]:
fraud_content_len = []

for message in messages:
    content = message["X-TIKA:content"].lstrip().strip()
    content_len = len(content.split())
    fraud_content_len.append(content_len)
    
content = email_dict['content']
content_char = []

for con in content:
    content_len = len(con.split())
    if content_len != 0:
        content_char.append(content_len)
        
    print(message['enron content summary'])
    print(message['enron capitilization value'])
    print(message['capitilzation ratio comparison'])
    print(message['enron domain lookup'])
    print(message['enron day created score'])
    print(message['enron day sent score'])